In [ ]:
import numpy as np
import os
import random
import cv2
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import wandb as wb
from wandb.keras import WandbCallback
import pandas as pd
import matplotlib.colors
import matplotlib.patches as patches
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import Callback
from matplotlib.backends.backend_pdf import PdfPages
from keras.utils.vis_utils import plot_model,model_to_dot

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

In [ ]:
os.mkdir('/kaggle/working/images')
os.mkdir('/kaggle/working/labels')
os.mkdir('/kaggle/working/rgb_label')

#os.mkdir('/kaggle/working/synth_images')
#os.mkdir('/kaggle/working/synth_labels')
#os.mkdir('/kaggle/working/synth_rgb_label')

In [ ]:
!git clone https://github.com/Vignesh-Desmond/image-segmentation-keras

In [ ]:
wb.login()

In [ ]:
run = wb.init(project='unet_random',
           config={
              "epochs": 50,
              "batch_size": 16,
               "steps_per_epoch":32,
              "loss_function": "WCCE",
              "architecture": "Resnet50_unet",
               "data":"Synthetic",
           },entity='splion360',name='Surya Prakash')
callbacks= [WandbCallback(data_type="image")]

In [ ]:
os.chdir('./image-segmentation-keras/')

In [ ]:
!python setup.py install

In [ ]:
os.chdir( "..")

In [ ]:
def batch_generator(dir_name1,dir_name2,img_size,dir_name3,batch_size = 50):
    
    image_file = random.sample(os.listdir(dir_name1),batch_size)
    
    for image in tqdm(image_file):
        img = Image.open(os.path.join(dir_name1,image))
        img = img.resize(img_size)
        img.save(os.path.join('/kaggle/working/images',image))
        
    for image in tqdm(image_file):
        img = Image.open(os.path.join(dir_name2, image[:-3]+'png'))
        img = img.resize(img_size)
        img.save(os.path.join('/kaggle/working/labels',image[:-3]+'png'))
    
        
    for image in tqdm(image_file):
        img = Image.open(os.path.join(dir_name3, image[:-3]+'png'))
        img = img.resize(img_size)
        img.save(os.path.join('/kaggle/working/rgb_label',image[:-3]+'png'))
       

    return 0

In [ ]:
original_image_file = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images'
label_img = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic'
rgb_label = '../input/semantic-drone-dataset/RGB_color_image_masks/RGB_color_image_masks'

batch_size = len(os.listdir(original_image_file))

In [ ]:
batch_generator(dir_name1=original_image_file,dir_name2 = label_img,dir_name3=rgb_label,img_size=(512,512),batch_size = 5)

In [ ]:
os.chdir( "./image-segmentation-keras/")

In [ ]:
!rm -rf '/kaggle/working/image-segmentation-keras'

In [ ]:
from keras_segmentation.models.segnet import resnet50_segnet

In [ ]:
model = resnet50_segnet(n_classes=24,input_height=512,input_width=512)

In [ ]:
df = pd.read_csv('../../input/semantic-drone-dataset/class_dict_seg.csv')

In [ ]:
y  = np.stack((df[' r'],df[' g'],df[' b']),axis=1)

In [ ]:
color_list = [tuple(i) for i in y]
class_list = list(df['name'])

In [ ]:
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [ ]:
color_hex = []
for rgb in color_list:
    hexc = rgb_to_hex(rgb)
    color_hex.append(hexc)

In [ ]:
from keras_segmentation.train import *
from keras_segmentation.predict import *

In [ ]:
image_file = '/kaggle/working/images'
label = '/kaggle/working/labels'
rgblabel = '/kaggle/working/rgb_labels'

In [ ]:
os.chdir('..')

In [ ]:
#Don't run this cell unless and otherwise a mistake is made in train_test_split
!rm -rf 'cvlabel' 'cvdata' 'trainlabel' 'traindata' 'testlabel' 'testdata' 'cv_rgblabel' 'train_rgblabel' 'test_rgblabel'

In [ ]:
os.mkdir('/kaggle/working/traindata')
os.mkdir('/kaggle/working/trainlabel')
os.mkdir('/kaggle/working/train_rgblabel')
os.mkdir('/kaggle/working/testdata')
os.mkdir('/kaggle/working/testlabel')
os.mkdir('/kaggle/working/test_rgblabel')

In [ ]:
def train_test_split(traindir,labeldir,rgblabeldir,ratio_split_test,ratio_split_train,want_cv = False,ratio_split_cv=0.2):
    
    
    size_train = int(len(os.listdir(traindir))*ratio_split_train)
    size_test = int(len(os.listdir(traindir))*ratio_split_test)
    size_cv = int(len(os.listdir(traindir))*ratio_split_cv) 
    
    if want_cv:
        trainlist = random.sample(sorted(os.listdir(traindir)),size_train)
        labeltrain = [image[:-3]+'png' for image in trainlist]

        cvlist = random.sample([image for image in sorted(os.listdir(traindir)) if image not in trainlist],size_cv)
        labelcv = [image[:-3]+'png' for image in cvlist]

        testlist = [image for image in sorted(os.listdir(traindir)) if image not in trainlist and image not in cvlist]
        labeltest = [image[:-3]+'png' for image in testlist]
        
        os.mkdir('/kaggle/working/cvdata')
        os.mkdir('/kaggle/working/cvlabel')
        os.mkdir('/kaggle/working/cv_rgblabel')
        
        
        for image in tqdm(trainlist):

            img = Image.open('/kaggle/working/images/'+image)
            img.save(os.path.join('/kaggle/working/traindata',image))

        for image in tqdm(testlist):
            img = Image.open('/kaggle/working/images/'+image)
            img.save(os.path.join('/kaggle/working/testdata',image))    

        for image in tqdm(labeltrain):
            img = Image.open('/kaggle/working/labels/'+image)
            img.save(os.path.join('/kaggle/working/trainlabel',image[:-3]+'png'))
          
        # Rgb label  
        for image in tqdm(labeltrain):
            img = Image.open('/kaggle/working/rgb_label/'+image[:-3]+'png')
            img.save(os.path.join('/kaggle/working/train_rgblabel',image[:-3]+'png'))

        for image in tqdm(labeltest):
            img = Image.open('/kaggle/working/labels/'+image[:-3]+'png')
            img.save(os.path.join('/kaggle/working/testlabel',image[:-3]+'png'))
        
        #Rgb label
        for image in tqdm(labeltest):
            img = Image.open('/kaggle/working/rgb_label/'+image[:-3]+'png')
            img.save(os.path.join('/kaggle/working/test_rgblabel',image[:-3]+'png'))
            
        for image in tqdm(cvlist):
            img = Image.open('/kaggle/working/images/'+image[:-3]+'.png')
            img.save(os.path.join('/kaggle/working/cvdata',image[:-3]+'png'))
        
        for image in tqdm(labelcv):
            img = Image.open('/kaggle/working/labels/'+image[:-3]+'png')
            img.save(os.path.join('/kaggle/working/cvlabel',image[:-3]+'png'))
        
        #Rgb label
        for image in tqdm(labelcv):
            img = Image.open('/kaggle/working/rgb_label/'+image[:-3]+'png')
            img.save(os.path.join('/kaggle/working/cv_rgblabel',image[:-3]+'png'))
            
        print(f"Number of training images and labels : {len(trainlist) , len(labeltrain)}")
        print(f"Number of cv images and labels : {len(cvlist) , len(labelcv)}")
        print(f"Number of test images and labels : {len(testlist) , len(labeltest)}")
        print()
        print("Directories created !!!")
            
            
            
    else:
        trainlist = random.sample(sorted(os.listdir(traindir)),size_train)
        labeltrain = ['label'+image[:-3]+'.png' for image in trainlist]
        
        testlist = [image for image in sorted(os.listdir(traindir)) if image not in trainlist]
        labeltest = ['label'+image[:-3]+'.png' for image in testlist]
        
        
        for image in tqdm(trainlist):

            img = Image.open('/kaggle/working/images/'+image)
            img.save(os.path.join('/kaggle/working/traindata',image))

        for image in tqdm(testlist):
            img = Image.open('/kaggle/working/images/'+image)
            img.save(os.path.join('/kaggle/working/testdata',image))    

        for image in tqdm(labeltrain):
            img = Image.open('/kaggle/working/labels/'+image)
            img.save(os.path.join('/kaggle/working/trainlabel','img'+image[5:]))
        
        for image in tqdm(labeltrain):
            img = Image.open('/kaggle/working/rgb_label/'+'rgb_label'+image[5:])
            img.save(os.path.join('/kaggle/working/test_rgblabel','rgb_label'+image[5:]))
            
    
        for image in tqdm(labeltest):
            img = Image.open('/kaggle/working/labels/'+image)
            img.save(os.path.join('/kaggle/working/testlabel','img'+image[5:]))
        
        for image in tqdm(labeltest):
            img = Image.open('/kaggle/working/rgb_label/'+'rgb_label'+image[5:])
            img.save(os.path.join('/kaggle/working/test_rgblabel','rgb_label'+image[5:]))


        print(f"Number of training images and labels : {len(trainlist) , len(labeltrain)}")
        print(f"Number of test images and labels : {len(testlist) , len(labeltrain)}")
        print()
        print("Directories created !!!")

In [ ]:
train_test_split(image_file,label,rgb_label,ratio_split_test=0.4,ratio_split_train=0.5,want_cv=True,ratio_split_cv=0.1) 

In [ ]:
os.chdir('./image-segmentation-keras/')

In [ ]:
os.mkdir('./checkpoint')

In [ ]:
#Don't run this cell unless and otherwise a mistake is made 
!rm -rf './checkpoint'

In [ ]:
train(model = model,
     train_images = '/kaggle/working/traindata',
     train_annotations = '/kaggle/working/trainlabel',
     verify_dataset = False,
     validate = True,
     steps_per_epoch=16,
     val_images='/kaggle/working/cvdata/',
     val_annotations='/kaggle/working/cvlabel/',
     epochs = 1,
      focal=True,
     batch_size = 5,
     checkpoints_path = '/kaggle/working/image-segmentation-keras/checkpoint/',
     do_augment=False,
    #callbacks = callbacks
     )

In [ ]:
run.finish()

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.to_csv('/kaggle/working/Losses.csv')

In [ ]:
evaluate(model, inp_images_dir="/kaggle/working/testdata", annotations_dir="/kaggle/working/testlabel")

In [ ]:
os.chdir('..')

In [ ]:
class_colors = color_n
labels = class_list

In [ ]:
!rm -rf '/kaggle/working/predicted_labels'

In [ ]:
color_n = []
for color in color_list:
    r,g,b = color
    new = (r/255.0,g/255.0,b/255.0) 
    color_n.append(new)

In [ ]:
os.mkdir('/kaggle/working/predicted_labels')

In [ ]:
## Visualisation of the output produced by the model

def visualisation_plot(dirtolook:str,dirtolooklabel:str,class_names,colors):
    
    cmp = matplotlib.colors.ListedColormap(
          color_n)
    index = 1  
    for img in tqdm(os.listdir('./'+dirtolook)):

        input_image = '/kaggle/working/'+ dirtolook +'/'+img
        out = model.predict_segmentation(
        inp=input_image,
        out_fname='out.png'
        )

        fig, axs = plt.subplots(1, 4, figsize=(12,12), constrained_layout=True)
         
        n_classes = len(class_names)
        legimg    = np.zeros(((len(class_names) * 25) + 46, 256, 3),
                     dtype="uint8") + 255
        
        
        class_names_colors = enumerate(zip(class_names[:n_classes],
                                       colors[:n_classes]))
        
        axs[0].imshow(legimg)
        axs[0].axis('off')
        
        for (i, (class_name, color)) in class_names_colors:
            axs[0].text(200, (i * 25) + 20,class_name,style='italic',fontsize=12,color='k',horizontalalignment='right')
            rect = patches.Rectangle((210, (i * 25)), 15, (i*25)+25-(i*25), linewidth=1,facecolor=color)
            axs[0].add_patch(rect)
       

        img_orig = plt.imread(input_image)
        axs[1].imshow(img_orig)
        axs[1].set_title('Original image')
        axs[1].grid(False)
        axs[1].axis('off')

        
        rgb_label     = img[:-3]+'png'
        img_rgb_label = os.path.join('/kaggle/working/rgb_label/',rgb_label)
        img_rgb_label = Image.open(img_rgb_label)
        img_rgb_label =  np.array(img_rgb_label)

        axs[2].imshow(img_rgb_label,cmap=cmp)
        axs[2].set_title('Ground truth')
        axs[2].grid(False)
        axs[2].axis('off')

        axs[3].imshow(out,cmap=cmp)
        axs[3].set_title('Predicted image')
        axs[3].grid(False)
        axs[3].axis('off')
        
        plt.savefig(os.path.join('/kaggle/working/predicted_labels/',f"img_{index}.png"))
        plt.close()
        index+=1

In [ ]:
visualisation_plot('testdata','test_rgblabel',labels,class_colors)

In [ ]:
!zip -r predicted_label.zip '/kaggle/working/predicted_labels'

In [ ]:
os.chdir('/kaggle/working')

**END OF IMPORTANT CODE. DONT EXECUTE THE REST!!!**